In [16]:
import pandas as pd
df = pd.read_pickle("original_dataframe.p")

In [5]:
from matbench.core.preprocess import Preprocess
from matbench.core.featurize import Featurize
from matbench.automl.tpot_utils import TpotAutoml
import warnings
warnings.filterwarnings("ignore")

#df = df[['structure', 'composition', 'formula', 'e_form']]
#df = df[[composition', 'formula', 'e_form']]
target_key ='e_form'

In [18]:
f = Featurize()
df = f.auto_featurize(df)
df.to_pickle("featurized_dataframe.p")

Dimensionality: 100%|██████████| 24759/24759 [4:49:22<00:00,  1.43it/s]


In [1]:
import pandas as pd
df = pd.read_pickle("featurized_dataframe.p")

In [3]:
from matbench.core.preprocess import Preprocess
p = Preprocess()
dfp = p.preprocess(df, "e_form", scale=True, max_na_frac=0.01)
dfp.to_pickle("processed_dataframe.p")

2018-09-16 17:09:43 INFO     Before handling na: 24759 samples, 596 features
2018-09-16 17:09:43 INFO     These 155 features were removed as they had more than 1.0% missing values:
{'mean hexagonal planar CN_6', 'std_dev trigonal pyramidal CN_4', 'mean wt CN_22', 'mean absolute deviation in relative bond length', 'mean wt CN_17', 'std_dev wt CN_13', 'mean T-shaped CN_3', 'std_dev hexagonal bipyramidal CN_8', 'mean q4 CN_10', 'mean ordering parameter shell 1', 'mean wt CN_5', 'std_dev q4 CN_10', 'std_dev body-centered cubic CN_8', 'mean pentagonal bipyramidal CN_7', 'mean wt CN_11', 'std_dev bent 120 degrees CN_2', 'range coefficient_of_linear_thermal_expansion', 'std_dev rectangular see-saw-like CN_4', 'std_dev wt CN_17', 'std_dev wt CN_21', 'std_dev trigonal bipyramidal CN_5', 'mean q6 CN_11', 'mean cuboctahedral CN_12', 'range neighbor distance variation', 'std_dev wt CN_24', 'mean q6 CN_12', 'mean ordering parameter shell 2', 'std_dev wt CN_1', 'mean body-centered cubic CN_8', 'std_

/global/u2/a/ardunn/tutorials/atomate_dev/codes/matbench/matbench/core/preprocess.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  number_df[number_cols] = MinMaxScaler().fit_transform(number_df)
/global/homes/a/ardunn/.conda/envs/PY3PRODUCTION/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfp.drop(target_key, axis=1).values, dfp[target_key], train_size=0.9, test_size=0.1)




In [7]:
tpot = TpotAutoml(mode="regression",
                  max_time_mins=360,
                  scoring='neg_mean_absolute_error',
                  feature_names=dfp.drop(target_key, axis=1).columns,
                  n_jobs=8,
                  verbosity=2)

In [8]:
tpot.fit(X_train, y_train)

Optimization Progress: 314pipeline [1:54:11, 141.14s/pipeline]

Generation 1 - Current best internal CV score: -0.13125802933676725


Optimization Progress: 463pipeline [2:43:35, 99.82s/pipeline]

Generation 2 - Current best internal CV score: -0.13125802933676725


Optimization Progress: 593pipeline [3:30:45, 95.22s/pipeline]

...

Optimization Progress: 730pipeline [4:17:17, 97.29s/pipeline]

Generation 4 - Current best internal CV score: -0.13125802933676725


Optimization Progress: 868pipeline [5:09:44, 83.23s/pipeline]

Generation 5 - Current best internal CV score: -0.13125802933676725



364.6132856166667 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RidgeCV(ExtraTreesRegressor(input_matrix, bootstrap=True, max_features=0.25, min_samples_leaf=7, min_samples_split=12, n_estimators=100))


In [10]:
top_scores = tpot.get_top_models(return_scores=True)
print('top cv scores:')
print(top_scores)
#print('top models')
#print(tpot.top_models)
test_score = tpot.score(X_test, y_test)
print('the best test score:')
print(test_score)

top cv scores:
OrderedDict([('RidgeCV', -0.13125802933676725), ('ExtraTreesRegressor', -0.1502514671770065), ('ElasticNetCV', -0.18110735481620627), ('LassoLarsCV', -0.1965264125539697), ('LinearSVR', -0.22109537633181037), ('DecisionTreeRegressor', -0.221868462226214), ('RandomForestRegressor', -0.2280188094830547), ('GradientBoostingRegressor', -0.3131813843914159), ('XGBRegressor', -0.39625256455075963), ('KNeighborsRegressor', -inf)])
the best test score:
-0.12182957304885537
